In [46]:
from fastapi import FastAPI, Request, UploadFile, File, Form, HTTPException
from fastapi.responses import JSONResponse, FileResponse
from fastapi.middleware.cors import CORSMiddleware
from fastapi.staticfiles import StaticFiles
import firebase_admin
from firebase_admin import credentials, firestore
import os
from datetime import datetime, timedelta
import json
import time

In [47]:
if not firebase_admin._apps:
    cred_path = os.environ.get('FIREBASE_CREDENTIALS')
    if cred_path:
        cred_dict = json.loads(cred_path)
        cred = credentials.Certificate(cred_dict)
    else:
        cred_path = os.environ.get('FIREBASE_CREDENTIALS_PATH', 'firebase-credentials.json')
        if os.path.exists(cred_path):
            cred = credentials.Certificate(cred_path)
        else:
            raise Exception("Firebase credentials not found. Set FIREBASE_CREDENTIALS or FIREBASE_CREDENTIALS_PATH")
    # Initialize the Firebase app using the constructed credentials
    firebase_admin.initialize_app(cred)

db = firestore.client()

In [48]:
def fetch_master_items():
    """Fetches the entire master checklist item list."""
    try:
        doc_ref = db.collection('config').document('checklist_items')
        doc = doc_ref.get()
        if doc.exists and 'items' in doc.to_dict():
            return doc.to_dict()['items']
        return []
    except Exception:
        return []

In [49]:
def fetch_all_last_completions():
    """Fetches the last completion date for each task across all dates (same as /api/checklist/last-completions)."""
    try:
        checklists_ref = db.collection('checklists')
        all_checklists = checklists_ref.stream()

        last_completions = {}  # {item_id: 'YYYY-MM-DD'}

        for checklist_doc in all_checklists:
            checklist_data = checklist_doc.to_dict()
            checked = checklist_data.get('checked', {})
            doc_date = checklist_doc.id

            for item_id, users_checked in checked.items():
                if users_checked:
                    if item_id not in last_completions or doc_date > last_completions[item_id]:
                        last_completions[item_id] = doc_date
        return last_completions
    except Exception:
        return {}

In [50]:
master_items = fetch_master_items()
last_completions = fetch_all_last_completions()

In [53]:
start_date = '2025-11-26'
end_date = '2025-11-26'
start_dt = datetime.strptime(start_date, '%Y-%m-%d')
end_dt = datetime.strptime(end_date, '%Y-%m-%d')

current_dt = start_dt
period_due_counts = {1:0,3:0,7:0,30:0,120:0}
items_due_count = 0

for item in master_items:
    # print(item)
    item_id = item.get('id')
    print(item_id)
    period_days = item.get('periodDays', 0)
    print(period_days)
    
    # # Check if the task is due for the current day based on recurrence
    is_due = True
    
    # # Apply Rule 3: Periodic filter (Only if periodDays > 0)
    if period_days is not None and period_days > 0:
        last_completion_date_str = last_completions.get(item_id)
        print(last_completion_date_str)
        if last_completion_date_str:
    #         # Calculate days since last completion
            last_date_dt = datetime.strptime(last_completion_date_str, '%Y-%m-%d')
    #         # Use 00:00:00 time to align with the front-end's date comparison
            days_since = (current_dt.replace(hour=0, minute=0, second=0, microsecond=0) - last_date_dt.replace(hour=0, minute=0, second=0, microsecond=0)).days
            print(days_since)
    #         # Hide task if NOT enough days have passed
            if days_since < period_days:
                is_due = False

    if is_due:
        items_due_count += 1
        period_due_counts[period_days] = period_due_counts.get(period_days, 0) + 1


item_2
1
2025-11-19
7
item_3
1
2025-11-19
7
item_4
7
2025-11-19
7
item_5
1
2025-11-19
7
item_6
1
2025-11-19
7
item_7
1
2025-11-19
7
item_8
1
2025-11-19
7
item_9
1
2025-11-19
7
item_10
3
2025-11-19
7
item_11
7
2025-11-19
7
item_12
7
2025-11-19
7
item_13
30
2025-11-19
7
item_14
90
2025-11-19
7
item_15
1
2025-11-19
7
item_16
30
2025-11-19
7
item_17
1
2025-11-19
7
item_18
1
2025-11-19
7
item_19
1
2025-11-19
7
item_20
1
2025-11-19
7
item_21
1
2025-11-19
7
item_22
1
2025-11-19
7
item_23
1
2025-11-19
7
item_24
30
2025-11-19
7
item_25
90
2025-11-19
7
item_26
1
2025-11-19
7
item_27
1
2025-11-19
7
item_28
1
2025-11-19
7
item_29
1
2025-11-19
7
item_30
1
2025-11-19
7
item_31
1
2025-11-19
7
item_32
1
2025-11-19
7
item_33
7
2025-11-19
7
item_34
3
2025-11-19
7
item_35
7
2025-11-19
7
item_36
30
2025-11-19
7
item_37
120
2025-11-19
7
item_38
1
2025-11-19
7
item_39
30
2025-11-19
7
item_40
1
2025-11-19
7
item_41
1
2025-11-19
7
item_42
1
2025-11-19
7
item_43
1
2025-11-19
7


In [54]:
period_due_counts

{1: 27, 3: 2, 7: 5, 30: 0, 120: 0}